## some dependent library for PETabc

One can check whole library in the requirement.txt


In [1]:
# !pip install torch
# !pip install numpy
# !pip install pandas
# !pip install torchdiffeq
# !pip install seaborn


In [2]:
## All the results are stored in the 'figure' folder.
# import some dependancy
import pandas as pd
import torch
from numpy import genfromtxt


# Run ABC by lpnt model

In [3]:

from PET_abc.lp_nt_PETabc import lp_nt_PETabc
from PET_abc.modelchoice_lpntPET import MC_abc_lpntPET

In [4]:
data_frame = pd.read_csv("ToTest-Real-1.csv")
data_frame.head()


Time(min)        Cr    Ctreal
0   1.850000  0.000000  0.000000
1   1.883333  0.000000  0.000000
2   1.916667  0.000000  0.000000
3   1.950000  0.000000  0.000000
4   1.983333  0.100355  0.492418

In [5]:
data_tensor = torch.tensor(data_frame.to_numpy(), device='cpu', dtype=torch.float64)
Ct = data_tensor[:, 2]
Cr = data_tensor[:, 1]
Ti = data_tensor[:, 0]

In [6]:
#Prior information
R1a = 0.0
R1b = 2.0
nnls = True

obj = lp_nt_PETabc(
    Ct, Cr, Ti, R1a=R1a, R1b=R1b, S=100000, h=0.01,nnls = nnls,Sequential=False)

result = MC_abc_lpntPET(Ct, Cr, Ti,obj,PLOT=True)


/opt/anaconda3/envs/PETabc/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
# For more information, one can use help to get the function document

help(lp_nt_PETabc)

Help on function lp_nt_PETabc in module PET_abc.lp_nt_PETabc:

lp_nt_PETabc(Ct, Cr, Ti, R1a=0.8, R1b=1.1, K2alpha=0.2, K2beta=0.6, K2a_alpha=0.1, K2a_beta=0.4, gamma_a=0, gamma_b=0.2, tD_a=15, tD_b=25, tP_b=40, alpha_a=0, alpha_b=3, k2a_per=100, k2a_int=[0.25, 0.75], h=0.05, S=100000, PLOT=True, nnls=False, Sequential=False, Seq=0)
    This function peforms ABC inference for the lp-nt model.
    Input Args:
    Ct vector of concentration values of the tracer in the target tissue.
    Cr vector of concentration values in the reference regions.
    Ti vector of time points of observations.
    S number of simulations. Default at 10^5.
    R1a lower bound of the uniform prior distribution for R1. Default at 0.5.
    R1a upper bound of the uniform prior distribution for R1. Default at 1.6.
    K2alpha lower bound of the uniform prior distribution for K2. Default at 0.
    K2beta upper bound of the uniform prior distribution for K2. Default at 1.
    K2a.alpha lower bound of the uniform pri

we can also use sequential ABC to iteratively update the posterior, here we show iterateively update the R1 in 5 round

In [8]:
Error_list = []
num_Seq = 5
for i in range(num_Seq):
    obj, Error2_rep = lp_nt_PETabc(
        Ct, Cr, Ti, R1a=R1a, R1b=R1b, S=100000, h=0.01, Sequential = True,Seq=i
    )
    Error_list.append(Error2_rep)
    R1a = obj["ABCout_act_accepted"][:, 0].min()
    R1b = obj["ABCout_act_accepted"][:, 0].max()

print("Finish!")

/Users/yk/Documents/PET/PET_ABC_parallel/PET_abc/lp_nt_PETabc.py:373: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(1, 1, figsize=(5, 5))
/Users/yk/Documents/PET/PET_ABC_parallel/PET_abc/lp_nt_PETabc.py:297: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  f, ax = plt.subplots(1, 3, figsize=(15, 5))


Finish!


# Run ABC

In [9]:
from PET_abc.abc import PETabc
from PET_abc.modelchoice_abc import MC_abc

In [10]:
my_data = genfromtxt('Data-Pois-12C-l1.csv', delimiter='')
data = torch.from_numpy(my_data[:,0])#choose one data
t = torch.arange(0,61).to(torch.float32)
obj1 = PETabc(data,tspan=t,PLOT=True)

#two tissue model

my_data = genfromtxt('Data-Pois-22C-l1.csv', delimiter='')

data = torch.from_numpy(my_data[:,0])
t = torch.arange(0,61).to(torch.float32)
obj2 = PETabc(data,tspan=t,model = 'two')  

result = MC_abc(data,tspan=t,abc_single=obj1,abc_two=obj2)

Simulate some data

In [ ]:
from PET_abc.function import TAC_One_Compartment,TAC_2_Compartment


sample_1_tissue = TAC_One_Compartment()
sample_2_tissue = TAC_2_Compartment()